In [17]:
import time
import math
import pickle
import pandas as pd
#import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import linear_model
from sklearn.calibration import calibration_curve 
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder, normalize
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import brier_score_loss, make_scorer , f1_score, accuracy_score, log_loss
from sklearn.model_selection import RandomizedSearchCV


In [18]:
filename_train = 'criminal_train.csv'
filename_test = 'criminal_test.csv'
trainall = pd.read_csv('{}'.format(filename_train),header=0)
test = pd.read_csv('{}'.format(filename_test),header=0)

In [19]:
print('Size of training set = ',trainall.shape)
print('Column names =', trainall.columns.values)
# independent variables
x = list(trainall.columns.values)[:-1]
# dependent variable
y = 'Criminal'
#dftrain.head()

Size of training set =  (45718, 72)
Column names = ['PERID' 'IFATHER' 'NRCH17_2' 'IRHHSIZ2' 'IIHHSIZ2' 'IRKI17_2' 'IIKI17_2'
 'IRHH65_2' 'IIHH65_2' 'PRXRETRY' 'PRXYDATA' 'MEDICARE' 'CAIDCHIP'
 'CHAMPUS' 'PRVHLTIN' 'GRPHLTIN' 'HLTINNOS' 'HLCNOTYR' 'HLCNOTMO' 'HLCLAST'
 'HLLOSRSN' 'HLNVCOST' 'HLNVOFFR' 'HLNVREF' 'HLNVNEED' 'HLNVSOR' 'IRMCDCHP'
 'IIMCDCHP' 'IRMEDICR' 'IIMEDICR' 'IRCHMPUS' 'IICHMPUS' 'IRPRVHLT'
 'IIPRVHLT' 'IROTHHLT' 'IIOTHHLT' 'HLCALLFG' 'HLCALL99' 'ANYHLTI2'
 'IRINSUR4' 'IIINSUR4' 'OTHINS' 'CELLNOTCL' 'CELLWRKNG' 'IRFAMSOC'
 'IIFAMSOC' 'IRFAMSSI' 'IIFAMSSI' 'IRFSTAMP' 'IIFSTAMP' 'IRFAMPMT'
 'IIFAMPMT' 'IRFAMSVC' 'IIFAMSVC' 'IRWELMOS' 'IIWELMOS' 'IRPINC3'
 'IRFAMIN3' 'IIPINC3' 'IIFAMIN3' 'GOVTPROG' 'POVERTY3' 'TOOLONG' 'TROUBUND'
 'PDEN10' 'COUTYP2' 'MAIIN102' 'AIIND102' 'ANALWT_C' 'VESTR' 'VEREP'
 'Criminal']


In [20]:
(trainall.Criminal.sum())/trainall.Criminal.count()

0.069447482392055643

In [21]:
# Divide into holdout and training sets
msk = np.random.rand(len(trainall)) < 0.8
train = trainall[msk]
hdout = trainall[~msk]

In [22]:
#define hyperparamter distribution

def model_params_dist(model_key):
    from scipy.stats import randint
    params_dists = {
        'RandomForest': {
             "n_estimators": [300,400],
             "max_depth": [3, None],
             "max_features": randint(1, 11),
             "min_samples_split": randint(2, 11),
             "min_samples_leaf": randint(1, 11),
             "bootstrap": [True, False],
             "criterion": ["gini", "entropy"],
             "random_state":[130],
             "n_jobs":[-1]
         },
#         'XGBoost': {
#             "n_estimators":[300,400],
#              "max_depth": [3, 5],
#              "learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
#              "booster": ['gbtree','gblinear','dart'],
#              "n_jobs":[-1],
#              "random_state":[130]
#          },
        'LogisticRegression': {
            'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,10000,100000]
             
         }
    }
    return params_dists[model_key]


# elect2 xgboost tuning grid
# {'max_depth': [1, 5, 10], 'max_delta_step': [1, 5, 10], 
#   'subsample': [0.5, 1], 'reg_lambda': [0, 0.5, 1], 'learning_rate': [0.1, 0.05], 
#   'reg_alpha': [0, 0.5, 1], 'colsample_bytree': [0.5, 1]}


def check_max_n_iter(dist):
    import numpy
    from sklearn.model_selection import ParameterGrid
    all_lists = numpy.all([not hasattr(v, "rvs")
                            for v in dist.values()])
    if all_lists:
        param_grid = ParameterGrid(dist)
        return len(param_grid)        
    return float('infinity')

In [24]:
model_key = 'RandomForest'

models_dir = {"RandomForest" : RandomForestClassifier(),
          "LogisticRegression": linear_model.LogisticRegression()}#,
          # "XGBoost" : xgb.XGBClassifier()}
    

if model_key == 'LogisticRegression':        
    # scale/ normalize the train and test sets separately
    train = normalize(train, norm='l1')
    hdout = normalize(hdout, norm='l1') 
    

    
clf = models_dir[model_key]     

# Tuning hyper parameters and training the model with best hyper-parameters
param_dist = model_params_dist(model_key)
k_fold =  StratifiedKFold(n_splits=10, shuffle=True)
k_fold.get_n_splits(train[x],train[y])

# set n_iter
n_iter = 20        
if check_max_n_iter(model_params_dist(model_key)) < n_iter:
    n_iter = check_max_n_iter(model_params_dist(model_key))


random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter, scoring='accuracy', cv=k_fold, n_jobs=-1
)
random_search.fit(train[x],train[y])

# Predictions using the model with best hyper-parameters
predictions = random_search.predict_proba(hdout[x])
probs = [e1[1] for e1 in predictions]
hdout2 = hdout
hdout['probs'] = probs
hdout2['predictions'] = hdout2['probs'].round(0)

best_params = random_search.best_params_      

metrics={
         'brier_score': brier_score_loss(hold2[y], hold2['probs']),
         'log_loss': log_loss(hold2[y], hold2['probs'])
}  

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


NameError: name 'hold2' is not defined

In [31]:
clf = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=None, max_features=8, min_samples_leaf=6,
                             min_samples_split=6, n_estimators=400, n_jobs=-1,random_state=130)
clf.fit(trainall[x],trainall[y])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=6,
            min_samples_split=6, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=-1, oob_score=False, random_state=130,
            verbose=0, warm_start=False)

In [35]:
#gbm = models_dir[model_key]
#gbm.set_params(*best_params.fit(trainall[x],trainall[y]))
#pickle.dump(gbm, open("model_object.pickle.dat", "wb"))
#predictions_score = random_search.predict_proba(test[x])
predictions_score = clf.predict_proba(test[x])
probs = [e1[1] for e1 in predictions_score]
Criminal = [1 if p>= 0.5 else 0 for p in probs]
test2 = test[['PERID']]
test2['probs'] = probs
test2['Criminal']= Criminal

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
test2[['PERID','Criminal']].to_csv('submit_rf_20180121.csv',header=1,index=False)
test2.to_csv('test_rf_20180121.csv',header=1,index=False)

In [34]:
test2

,PERID,probs,Criminal
0,66583679,0.118617,0
1,35494679,0.000375,0
2,79424679,0.000081,0
3,11744679,0.001757,0
4,31554679,0.001577,0
5,83164679,0.000683,0
6,58474679,0.012711,0
7,40695779,0.231251,0
8,39315779,0.004966,0
9,58415779,0.093320,0
